In [1]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import math
import pickle

In [2]:
### Import data
#### 'flights.csv' can be downloaded on https://www.kaggle.com/datasets/usdot/flight-delays?select=flights.csv
df = pd.read_csv('flights.csv', parse_dates={"date":['YEAR', 'MONTH', 'DAY']}) #Load data per day
dM = pd.read_csv('flights.csv', parse_dates={"date":['YEAR', 'MONTH']}) # Load data per month
DY = pd.read_csv("flights.csv", parse_dates={"date":["YEAR"]}) #Load data per year

C:\Users\hugoo\AppData\Local\Temp\ipykernel_7640\2379387243.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flights.csv', parse_dates={"date":['YEAR', 'MONTH', 'DAY']}) #Load data per day
C:\Users\hugoo\AppData\Local\Temp\ipykernel_7640\2379387243.py:4: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dM = pd.read_csv('flights.csv', parse_dates={"date":['YEAR', 'MONTH']}) # Load data per month
C:\Users\hugoo\AppData\Local\Temp\ipykernel_7640\2379387243.py:5: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  DY = pd.read_csv("flights.csv", parse_dates={"date":["YEAR"]}) #Load data per year


In [3]:
df['date'] = df['date'].astype(str) #convert data to be handled
df["count"] = np.ones(len(df)) #create extra column
dM['date'] = dM['date'].astype(str) #convert data to be handled
dM["count"] = np.ones(len(dM)) #create extra column
dM['ORIGIN_AIRPORT'] = dM['ORIGIN_AIRPORT'].astype(str) #convert all airport formats to the same type - string
DY['date'] = DY['date'].astype(str) #convert data to be handled
DY["count"] = np.ones(len(DY)) #create extra column
DY['ORIGIN_AIRPORT'] = DY['ORIGIN_AIRPORT'].astype(str) #convert all airport formats to the same type - string

In [4]:
#Arrival delay for airline,
AL = df.groupby(["date", "AIRLINE"]).sum().reset_index() #Group data by day and Airline
AL["ArvCount"] = AL["ARRIVAL_DELAY"] / AL["count"] # calculate arrival delay per day per Airline
AL2 = dM.groupby(["date", "AIRLINE"]).sum().reset_index() #group data by month and Airline
AL2["ArvCount"] = AL2["ARRIVAL_DELAY"] / AL2["count"] #calculate airival delay per month per Airline

# Departure delay for airport
AP = dM.groupby(["date", "ORIGIN_AIRPORT"]).sum().reset_index() #Group data by month and Airport
AP["DepCount"] = AP["DEPARTURE_DELAY"] / AP["count"] #calculate departure delay per month per Airport
YP = DY.groupby(["date", "ORIGIN_AIRPORT"]).sum().reset_index() #Group data by month and Airport
YP["DepCount"] = AP["DEPARTURE_DELAY"] / AP["count"]#calculate departure delay per month per Airport

In [5]:
# Filling the gaps - Month
Month_unique = dM.date.unique() #Setting required inputs
Airline_unique = dM.AIRLINE.unique() #Setting required inputs
a = []
b = []
for i in range(len(Month_unique)):
    for j in range(len(Airline_unique)):
        a.append(Month_unique[i]) #fill list with Months
        b.append(Airline_unique[j]) #fill list with Airlines
c = np.zeros(len(a)) #All non-existing values will be 0, as they will not be overruled
data = {'Date': a,
        'Airline': b,
       'Average arrival delay': c} #create dictionairy
Months_df = pd.DataFrame(data) #convert to DataFrame


for k in range(len(Months_df.Date)): #fill the new dataframe with new real data
    for l in range(len(AL2.date)):
        if Months_df.Date[k]==AL2.date[l] and Months_df.Airline[k]==AL2.AIRLINE[l]:
            Months_df["Average arrival delay"][k] = AL2.ArvCount[l]

C:\Users\hugoo\AppData\Local\Temp\ipykernel_7640\514335400.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Months_df["Average arrival delay"][k] = AL2.ArvCount[l]


In [6]:
Days_unique = df.date.unique() #Setting required inputs
Airline_unique = df.AIRLINE.unique() #Setting required inputs
a = [] #Create new list
b = [] #create new list
for i in range(len(Days_unique)):
    for j in range(len(Airline_unique)):
        a.append(Days_unique[i]) #fill list with Days
        b.append(Airline_unique[j]) #fill list with airlines
c = np.zeros(len(a)) #All non-existing values will be 0, as they will not be overruled
data = {'Date': a,
        'Airline': b,
       'Average arrival delay': c} #create dictionairy
Days_df = pd.DataFrame(data) #convert to DataFrame


for k in range(len(Days_df.Date)): #fill the new dataframe with new real data
    for l in range(len(AL.date)):
        if Days_df.Date[k]==AL.date[l] and Days_df.Airline[k]==AL.AIRLINE[l]:
            Days_df["Average arrival delay"][k] = AL.ArvCount[l]   

C:\Users\hugoo\AppData\Local\Temp\ipykernel_7640\250150140.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Days_df["Average arrival delay"][k] = AL.ArvCount[l]


In [7]:
Months_df.to_pickle("./M_Airline.pkl") #store Dateframe
Days_df.to_pickle("./D_Airline.pkl") #store Dataframe

In [9]:
Months_Airline = pd.read_pickle("./M_Airline.pkl")
AA = Months_Airline.groupby(["Airline"]).mean() #average yearly arrival delay
display(AA)

,Average arrival delay
Airline,
AA,4.007417
AS,-1.017034
B6,6.590284
DL,0.214835
EV,6.307559
F9,12.893491
HA,1.984102
MQ,5.429764
NK,14.186793


In [10]:
#Filling the gaps - Month
Month_unique = dM.date.unique() #Setting required inputs
Airport_unique = dM.ORIGIN_AIRPORT.unique() #Setting required inputs
a = []
b = []
for i in range(len(Month_unique)):
    for j in range(len(Airport_unique)):
        a.append(Month_unique[i]) #list with months
        b.append(Airport_unique[j]) #list with airports
c = np.zeros(len(a)) #All non-existing values will be 0, as they will not be overruled within the next loop
data = {'Date': a, 
        'Airport': b,
       'Average departure delay': c} #create dictionairy
Months_DP = pd.DataFrame(data) #convert to dataframe


for k in range(len(Months_DP.Date)): #fill the new dataframe with new real data
    for l in range(len(AP.date)):
        if Months_DP.Date[k]==AP.date[l] and Months_DP.Airport[k]==AP.ORIGIN_AIRPORT[l]:
            Months_DP["Average departure delay"][k] = AP.DepCount[l]

C:\Users\hugoo\AppData\Local\Temp\ipykernel_7640\524986807.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Months_DP["Average departure delay"][k] = AP.DepCount[l]


In [11]:
Months_DP.to_pickle("./M_Airport.pkl")

In [12]:
#Filling the gaps - Year
YEAR_unique = DY.date.unique() #Setting required inputs
Airport_unique = DY.ORIGIN_AIRPORT.unique() #Setting required inputs
a = []
b = []
for i in range(1):
    for j in range(len(Airport_unique)):
        a.append(YEAR_unique[i]) #list with months
        b.append(Airport_unique[j]) #list with airports
c = np.zeros(len(a)) #All non-existing values will be 0, as they will not be overruled within the next loop
data = {'Date': a, 
        'Airport': b,
       'Average departure delay': c} #create dictionairyY
Year_DP = pd.DataFrame(data) #convert to dataframe


for k in range(len(Year_DP.Date)): #fill the new dataframe with new real data
    for l in range(len(YP.date)):
        if Year_DP.Date[k]==YP.date[l] and Year_DP.Airport[k]==YP.ORIGIN_AIRPORT[l]:
            Year_DP["Average departure delay"][k] = YP.DepCount[l]

C:\Users\hugoo\AppData\Local\Temp\ipykernel_7640\808115557.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Year_DP["Average departure delay"][k] = YP.DepCount[l]


In [13]:
Year_DP.to_pickle("./Y_Airport.pkl")